In [1]:
import pandas as np

In [2]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("yushuang/fin.highlight")

In [3]:
run_result = []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    this_run_data = dict()
    this_run_data.update(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    this_run_data.update(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    this_run_data.update({'name': run.name})

    this_run_data.update({'args': ' '.join(run.metadata['args'])})


    slot_to_remove = []
    for k, v in this_run_data.items():
        if type(v) not in [int, str, float, bool]:
            slot_to_remove.append(k)
    for s in slot_to_remove:
        del this_run_data[s]
    run_result.append(this_run_data)

    
runs_df = pd.DataFrame(run_result)
# runs_df['_timestamp'] = runs_df['_timestamp'].map(datetime_wrapper)
# runs_df.to_csv("project.csv")

In [4]:
runs_df.tail()['args']

167    --output_dir ./checkpoints/cnc_train_complex -...
168    --output_dir ./checkpoints/cnc_train_complex -...
169    --output_dir ./checkpoints/cnc_train_harsh --r...
170    --output_dir ./checkpoints/cnc_train_harsh --r...
171    --output_dir ./checkpoints/cnc_train_harsh --r...
Name: args, dtype: object

In [5]:
def check_if_resume_from_checkpoints(args_str):
    args = args_str.split()
    # print(args)
    if '--resume_from_checkpoint' in args:
        return True
    else:
        return False

In [6]:
runs_df['resume_from_checkpoint'] = runs_df['args'].map(check_if_resume_from_checkpoints)

In [7]:
sum(runs_df['resume_from_checkpoint'])

18

In [8]:
columns_to_use = [
    'name',
    'metric_for_best_model',
    '_timestamp'
]
metrics_names = ['f1', 'disorder', 'auprc', 'r_precision']
for m in metrics_names:
    for col in runs_df.columns:
        if m in col:
            columns_to_use.append(col)

In [14]:
show_metrics = ['disorder', 'f1', 'auprc', 'r_precision']
mbm_metrics = 'valid_f1'

def datetime_wrapper(integar):
    return pd.to_datetime(integar, utc=True, unit='s').strftime('%Y%m%d %H:%M:%S')

show_columns = ['name', 'metric_for_best_model']#, 'greater_is_better']
for show_metric in show_metrics:
    for split in ['train', 'valid', 'test', 'expert']:
        show_columns.append(f'eval/{split}_{show_metric}')
show_columns.append('_timestamp')
tmp = runs_df[show_columns].copy().dropna()
tmp['_timestamp'] = tmp['_timestamp'].map(datetime_wrapper)
tmp[tmp['metric_for_best_model'] == mbm_metrics].to_csv('disorder.csv')

In [10]:
show_metric = 'f1'
mbm_metrics = 'valid_disorder'

def datetime_wrapper(integar):
    return pd.to_datetime(integar, utc=True, unit='s').strftime('%Y%m%d %H:%M:%S')

show_columns = ['name', 'metric_for_best_model']#, 'greater_is_better']
for split in ['train', 'valid', 'test', 'expert']:
    show_columns.append(f'eval/{split}_{show_metric}')
show_columns.append('_timestamp')
tmp = runs_df[runs_df['resume_from_checkpoint'] == True][show_columns].copy().dropna()
tmp['_timestamp'] = tmp['_timestamp'].map(datetime_wrapper)
tmp[tmp['metric_for_best_model'] == mbm_metrics]

,name,metric_for_best_model,eval/train_f1,eval/valid_f1,eval/test_f1,eval/expert_f1,_timestamp
34,iterate_strict-naive,valid_disorder,0.373957,0.398229,0.318602,0.268590,20250408 02:11:21
43,iterate_strict-loose,valid_disorder,0.735642,0.476599,0.399477,0.325784,20250408 07:01:03
52,iterate_naive-strict,valid_disorder,0.226073,0.316474,0.234557,0.202150,20250408 13:14:59
73,iterate_naive-loose,valid_disorder,0.712860,0.482212,0.394925,0.342112,20250408 23:57:41
91,iterate_loose-strict,valid_disorder,0.325357,0.406611,0.321409,0.295933,20250409 05:08:44
103,iterate_loose-naive,valid_disorder,0.375625,0.389596,0.307129,0.255295,20250409 09:53:56


In [11]:
runs_df[runs_df['name'] == 'iterate_strict-naive'][['name']]

,name
27,iterate_strict-naive
28,iterate_strict-naive
30,iterate_strict-naive
31,iterate_strict-naive
33,iterate_strict-naive
34,iterate_strict-naive
